[View in Colaboratory](https://colab.research.google.com/github/ErikPartridge/machine-translation/blob/master/ENFR.ipynb)

In [1]:
!wget http://www.statmt.org/europarl/v7/fr-en.tgz

--2018-07-05 15:24:27--  http://www.statmt.org/europarl/v7/fr-en.tgz
Resolving www.statmt.org (www.statmt.org)... 129.215.197.184
Connecting to www.statmt.org (www.statmt.org)|129.215.197.184|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 202718517 (193M) [application/x-gzip]
Saving to: ‘fr-en.tgz’

fr-en.tgz            11%[=>                  ]  21.65M   384KB/s    eta 6m 47s 

fr-en.tgz            99%[==================> ] 193.31M   551KB/s    eta 1s     

fr-en.tgz           100%[===================>] 193.33M   551KB/s    in 7m 1s   

2018-07-05 15:31:28 (470 KB/s) - ‘fr-en.tgz’ saved [202718517/202718517]



In [2]:
!tar -xvf fr-en.tgz

europarl-v7.fr-en.en
europarl-v7.fr-en.fr


In [0]:
!sed -i '/^</ d' europarl-v7.fr-en.en 
!sed -i '/^</ d' europarl-v7.fr-en.fr
!sed -i '/^$/d' europarl-v7.fr-en.fr
!sed -i '/^$/d' europarl-v7.fr-en.en

In [0]:
with open ("europarl-v7.fr-en.fr", "r") as frf:
  fr = frf.readlines()
    
with open ("europarl-v7.fr-en.en", "r") as enf:
  en = enf.readlines()

In [0]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import text_to_word_sequence

frt = []
for line in fr:
  frt.append(one_hot(line, 12000))

ent = []
for line in en:
  ent.append(one_hot(line, 12000))

Using TensorFlow backend.


In [0]:
import numpy as np

frv = []
for line in frt:
  tmp = []
  for num in line:
    l = np.zeros(12000)
    l[num] = 1
    tmp.append(l)
  frv.append(np.array(tmp))
frv = np.array(frv)

envr = []
for line in ent:
  tmp = []
  for num in line:
    l = np.zeros(12000)
    l[num] = 1
    tmp.append(l)
  envr.append(np.array(tmp))
envr = np.array(envr)

In [4]:
from keras.models import Model
from keras.layers import Dense, Activation, Concatenate, GRU

inputs = Input(shape=(12000,40))
pos = Dense(512)(inputs)
pos = GRU(256)(pos)
pos = GRU(128)(pos)
pos = Dense(40)(pos)

ctx = Dense(512)(inputs)
ctx = LSTM(512)(inputs)
ctx = Dense(128)(inputs)

combined = Concatenate()(inputs, pos, ctx)
combined = LSTM(256, bidirectional=True)(combined)
combined = Dense(50)(combined)


model = Model(inputs=inputs, outputs=combined)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(data, labels)

Using TensorFlow backend.
